# Script #1 

- Takes in Sharepoint Export `OTS_RSVP_45.xlsx`
- Renames column headers to match the Knack Reservation schema `banners_reservations_ots.csv`
- Exports new Reservations table to `output/reservations.csv`
- Makes a new Reservations Locations Table to match schema from `banner_locations.csv`
- Exports new Reservations Locations to `output/reservation_locations.csv`

In [1]:
import pandas as pd
from pandas import read_excel
import numpy as np

df = pd.read_excel("./source_data/OTS_RSVP_45.xlsx", sheet_name="owssvr.dll")

In [2]:
# Setup a new dataframe for Reservations:
df_reservations = df

# Rename columns to match Knack schema
# Sharepoint: OTS_RSVP_45 column names of left
# Knack: banner_reservations_ots on right
column_names = {
    # Sharepoint : Knack
    'ID': 'Sharepoint ID',
    'Permit #': 'Permit Number',
    'EventName': 'Event Name OTS', 
    'Organization': 'Organization Name', 
    # 'Payment Rec?'
    'Delivered?': 'Banners Delivered',
    'Status': 'Banner Status OTS',
    'Quantity': 'Number of Banners OTS',
    # Count Banner Jobs
    'RSVP Date': 'CREATED_DATE',
    'Installation_Date': 'Installation Date OTS',
    'Removal_Date': 'Removal Date OTS',
    # Keep all 'Banner_Face' & 'OTS_Linker' 1-4 columns the same
    # 'App Received?'
    # 'Co-Sponsor?'
    # 'Design Rec?'
    'Will Pick-up?': 'Will Pick Up'
    # 'Item Type'
    # 'Path'
}
df_reservations.rename(columns = column_names, inplace= True)

# TODO: Confirm if we should we remove these unneeded columns?
removed_columns = [
#     'Payment Rec?',
#     'App Received?',
#     'Co-Sponsor?',
#     'Design Rec?',
    'Item Type',
    'Path'
]
df_reservations = df_reservations.drop(columns=removed_columns)



In [3]:
# Export Reservations table to CSV
df_reservations.to_csv(r'./output_data/reservations.csv', index=None, header=True)

# Script #2

### Inputs:
- Sharepoint reservations table `df_reservations`
- Reservation table `source_data/banner_reservations_ots_20190325.csv` w/ Knack given "Banner OTS ID"
- Reservation Location Table Schema `output_data/reservation_locations.csv`
- Location Table Schema `source_data/banners_over_the_street_locations.csv`

### Outputs:
- creates a new Reservation Locations Table
    - with a foreign key column of Knack ID of Reservation
    - with a "Location Name" column matching Location Table "Linker Label" as identifier 

In [4]:
# Import Reservation Table. We'll use this data to add "banner reservations OT" 
# data to the Reservation Locations dataframe using the "Banner OTD ID" column.
# 
# NOTE: banner_reservations_ots_20190325 had duplicate columns so I manually removed 
# the "Number of Banners OTS" col and resaved as "banner_reservations_ots_20190325_2.csv"
knack_reservations_df = pd.read_csv('./source_data/banner_reservations_ots_20190325_2.csv')

# Merge Banner OTS ID column to original Sharepoint Export
sharepoint_reservations_df = df_reservations

for i in sharepoint_reservations_df.index:
    # Give the Sharepoint rsvp Table Banner OTS IDs that match the Reservations Table exported from Knack
    banner_id = knack_reservations_df.loc[ knack_reservations_df["Sharepoint ID"] == sharepoint_reservations_df["Sharepoint ID"], 'Banner OTS ID']
    sharepoint_reservations_df["banner reservations OT"] = banner_id

In [5]:
# Create a new dataframe for Reservations Locations 
# using headers from the csv
reso_location_column_names = pd.read_csv('./source_data/banner_locations.csv').columns
reso_locations_df = pd.DataFrame(columns=reso_location_column_names)
reso_locations_df['Parent ID'] = None

In [6]:
locations_df = pd.read_csv('./source_data/banners_over_the_street_locations.csv')

In [18]:
import math

for i in df_reservations.index:
    has_one_location = df_reservations["Number of Banners OTS"][i] == 1
        
    if has_one_location:
        # Get Location value
        linker_location_series = locations_df.loc[ locations_df["Linker Name"] == df_reservations['OTS_Linker_1'][i], 'Linker Label']
        if len(linker_location_series) > 0: 
            linker_location = linker_location_series.values[0]
        else:
            linker_location = None
        
        # Change outdated LAMAR #8 address
        if df_reservations[f'OTS_Linker_1'][i] == '# 8: 4180 1/2 N LAMAR BLVD':
              linker_location = '# 8: 4199 1/2 N LAMAR BLVD (N - S)'
              linker_name = '# 8: 4199 1/2 N LAMAR BLVD'
        else:
            linker_name = df_reservations['OTS_Linker_1'][i]
        
        # TODO: Figure out why this one didn't take!
        # Change LAMAR #2 address
        if df_reservations[f'OTS_Linker_1'][i]  == '# 2: 9411 1/2 N LAMAR BLVD':
              linker_location = '#2: 9411 1/2 N LAMAR BLVD (N - S)'
              linker_name = '# 2: 9411 1/2 N LAMAR BLVD'
        else:
            linker_name = df_reservations[f'OTS_Linker_1'][i]
        
        data = {
            'banner reservations OT': df_reservations["banner reservations OT"][i],
            'Permit Number': df_reservations["Permit Number"][i],
            'banners_organization': df_reservations["Organization Name"][i],
            'Organization Name': df_reservations['Organization Name'][i],
            'Linker Location': linker_location,
            'Linker Name': df_reservations[f'OTS_Linker_1'][i],
            'Direction Facing': df_reservations['Banner_Face_1'][i],
            'OTS Install Date ': df_reservations['Installation Date OTS'][i],
            'OTS Remove Date': df_reservations['Removal Date OTS'][i],
            'Count Total': df_reservations['Number of Banners OTS'][i],
            'Status OTS': df_reservations['Banner Status OTS'][i],
        }
    
        # Create a new row
        row_series = pd.Series(data)
        # Add row to dataframe at the current iterator index
        reso_locations_df.loc[i] = row_series

    else:        
        x = 0
        while x < df_reservations["Number of Banners OTS"][i]:
            x += 1
            
            try:
                float(df_reservations["Permit Number"][i])
                permit_number = None
                parent_id = None
                
            except ValueError:
                parent_id = df_reservations["Permit Number"][i].split(" Permits ")[0].split("Parent ID")[1].strip().rstrip(',')
                permit_list = df_reservations["Permit Number"][i].split(" Permits ")[1].split(",")
                permit_number = permit_list[x - 1].strip()

            
            # This if is for an edge case were the row doesn't have a Permit Numbers
            # This seems to happen when the org for a banner is ATD.
#             if type(df_reservations["Permit Number"][i]) == float and math.isnan(df_reservations["Permit Number"][i]):
#                 permit_number = None
#                 parent_id = None
                
#             else:
#                 parent_id = df_reservations["Permit Number"][i].split(" Permits ")[0].split("Parent ID")[1].strip().rstrip(',')
#                 permit_list = df_reservations["Permit Number"][i].split(" Permits ")[1].split(",")
                
#                 permit_number = permit_list[x - 1].strip()
            
            # Get Location value
            linker_location_series = locations_df.loc[ locations_df["Linker Name"] == df_reservations[f'OTS_Linker_{x}'][i], 'Linker Label']
                  
            if len(linker_location_series) > 0: 
                linker_location = linker_location_series.values[0]
            else:
                linker_location = None
            
            # Change outdated LAMAR #8 address
            if df_reservations[f'OTS_Linker_{x}'][i] == '# 8: 4180 1/2 N LAMAR BLVD':
                  linker_location = '# 8: 4199 1/2 N LAMAR BLVD (N - S)'
                  linker_name = '# 8: 4199 1/2 N LAMAR BLVD'
            else:
                linker_name = df_reservations[f'OTS_Linker_{x}'][i]
            
            # Change LAMAR #2 address
            if df_reservations[f'OTS_Linker_{x}'][i]  == '# 2: 9411 1/2 N LAMAR BLVD':
                  linker_location = '#2: 9411 1/2 N LAMAR BLVD (N - S)'
                  linker_name = '# 2: 9411 1/2 N LAMAR BLVD'
            else:
                linker_name = df_reservations[f'OTS_Linker_{x}'][i]
                  
        
            data = {
                'banner reservations OT': df_reservations["banner reservations OT"][i],
                'Permit Number': permit_number,
                'banners_organization': df_reservations["Organization Name"][i],
                'Organization Name': df_reservations['Organization Name'][i],
                'Linker Location': linker_location,
                'Linker Name': linker_name,
                'Direction Facing': df_reservations[f'Banner_Face_{x}'][i],
                'OTS Install Date ': df_reservations['Installation Date OTS'][i],
                'OTS Remove Date': df_reservations['Removal Date OTS'][i],
                'Count Total': df_reservations['Number of Banners OTS'][i],
                'Status OTS': df_reservations['Banner Status OTS'][i],
                'Parent ID': parent_id
            }
            

            # Create a new row
            row_series = pd.Series(data)
            # Add row to dataframe at the current iterator index
            reso_locations_df.loc[f'{i}0{x}'] = row_series               

In [19]:
reso_locations_df

,Banner Job ID,Permit Number,banner reservations OT,banners_reservations_LPB,banners_district_sub_district,banners_organization,street_banners_comment,Organization Name,Linker Location,Linker Name,...,Count Total,Status OTS,Districts,Subdistricts,LPB Install Date,LPB Remove Date,Status LPB,Banner Location Comment,Banner Job ID.1,Parent ID
0,NaN,18-117963,1,NaN,NaN,"Service Dogs, Inc.",NaN,"Service Dogs, Inc.",#W: 3652 WESTBANK DR. (N - S),#W: 3652 WESTBANK DR.,...,1,Reservation Input,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,19-029392,2,NaN,NaN,Spark Cognition,NaN,Spark Cognition,#61: 200 1/2 S CONGRESS AVE (N - S),#61: 200 1/2 S CONGRESS AVE,...,1,Reservation Input,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201,NaN,18-117958,3,NaN,NaN,"Service Dogs, Inc.",NaN,"Service Dogs, Inc.",#32: 1799 1/2 S CONGRESS AVE (N - S),#32: 1799 1/2 S CONGRESS AVE,...,3,Reservation Input,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11982651
202,NaN,18-117959,3,NaN,NaN,"Service Dogs, Inc.",NaN,"Service Dogs, Inc.",#18: 2706 1/2 S LAMAR BLVD (N - S),#18: 2706 1/2 S LAMAR BLVD,...,3,Reservation Input,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11982651
203,NaN,18-117962,3,NaN,NaN,"Service Dogs, Inc.",NaN,"Service Dogs, Inc.",#15: 1502 1/2 W 5th ST (W),#15: 1502 1/2 W 5th ST,...,3,Reservation Input,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11982651
3,NaN,18-137917,4,NaN,NaN,Austin NARI,NaN,Austin NARI,#W: 3652 WESTBANK DR. (N - S),#W: 3652 WESTBANK DR.,...,1,Reservation Input,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
401,NaN,19-009296,5,NaN,NaN,Blanton Museum of Art,NaN,Blanton Museum of Art,# 5: 201 1/2 E MLK BLVD (E - W),# 5: 201 1/2 E MLK BLVD,...,2,Reservation Input,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12119979
402,NaN,19-009299,5,NaN,NaN,Blanton Museum of Art,NaN,Blanton Museum of Art,#64: 2100 1/2 E 7th ST (E - W),#64: 2100 1/2 E 7th ST,...,2,Reservation Input,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12119979
501,NaN,18-135333,6,NaN,NaN,Austin NARI,NaN,Austin NARI,#24: 2800 1/2 S LAMAR BLVD (N - S),#24: 2800 1/2 S LAMAR BLVD,...,3,Reservation Input,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12002659
502,NaN,18-135342,6,NaN,NaN,Austin NARI,NaN,Austin NARI,#16: 1206 1/2 W 38th ST (E - W),#16: 1206 1/2 W 38th ST,...,3,Reservation Input,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12002659


In [20]:
df_reservations

,Sharepoint ID,Permit Number,Event Name OTS,Organization Name,Payment Rec?,Banners Delivered,Banner Status OTS,Number of Banners OTS,CREATED_DATE,Installation Date OTS,...,OTS_Linker_2,Banner_Face_3,OTS_Linker_3,Banner_Face_4,OTS_Linker_4,App Received?,Co-Sponsor?,Design Rec?,Will Pick Up,banner reservations OT
0,566,18-117963,Mighty Texas Dog Walk,"Service Dogs, Inc.",Y,Y,Reservation Input,1,2018-09-14 14:45:08,2019-03-04,...,NaN,NaN,NaN,NaN,NaN,Y,N,Y,Y,1
1,605,19-029392,Time Machine 2019,Spark Cognition,N,Y,Reservation Input,1,2019-02-25 14:00:46,2019-03-04,...,NaN,NaN,NaN,NaN,NaN,Y,N,Y,Y,2
2,568,"Parent ID 11982651 Permits 18-117958, 18-11795...",Mighty Texas Dog Walk,"Service Dogs, Inc.",Y,Y,Reservation Input,3,2018-09-26 10:21:04,2019-03-11,...,#18: 2706 1/2 S LAMAR BLVD,East,#15: 1502 1/2 W 5th ST,NaN,NaN,Y,N,Y,Y,3
3,556,18-137917,Tour of Remodeled Homes,Austin NARI,Y,N,Reservation Input,1,2018-08-15 07:43:15,2019-03-18,...,NaN,NaN,NaN,NaN,NaN,Y,N,Y,Y,4
4,588,"Parent ID 12119979, Permits 19-009296, 19-009299",Blanton Block Party,Blanton Museum of Art,Y,N,Reservation Input,2,2019-01-16 08:35:21,2019-03-18,...,#64: 2100 1/2 E 7th ST,NaN,NaN,NaN,NaN,Y,N,Y,Y,5
5,555,"Parent ID 12002659 Permits 18-135333, 18-13534...",Tour of Remodeled Homes,Austin NARI,Y,N,Reservation Input,3,2018-08-09 12:21:12,2019-03-25,...,#16: 1206 1/2 W 38th ST,South,#61: 200 1/2 S CONGRESS AVE,NaN,NaN,Y,N,Y,Y,6
6,560,18-160473,Austin Dance Frestival,Kathy Dunn Hamrick Dance Co.,Y,N,Reservation Input,1,2018-09-10 07:33:25,2019-03-25,...,NaN,NaN,NaN,NaN,NaN,Y,N,Y,Y,7
7,593,"Parent ID 12131692 Permits 19-019309, 19-01931...",Statesman Cap10k,Austin American-Statesman,Y,N,Reservation Input,3,2019-02-05 13:57:50,2019-03-25,...,# 7: 1216 1/2 S LAMAR BLVD,West,#15: 1502 1/2 W 5th ST,NaN,NaN,Y,N,Y,Y,8
8,596,19-024603,All-Star Sports Summer Camp,Crow Luna Foundation,Y,Y,Reservation Input,1,2019-02-08 14:58:04,2019-03-25,...,NaN,NaN,NaN,NaN,NaN,Y,N,N,Y,9
9,604,"Parent ID 12139991, Permits 19-026366, 19-0263...",Texas Football Orange-White Spring Game,UT - Athletics,N,Y,Reservation Input,3,2019-02-19 15:28:50,2019-04-01,...,# 5: 201 1/2 E MLK BLVD,North,NaN,NaN,NaN,Y,N,N,Y,10


In [91]:
# Export Reservations Locations table to CSV
reso_locations_df.to_csv(r'./output_data/reservation_locations.csv', index=None, header=True)